In [1]:
from dcicutils import ff_utils
from functions.wfr import *
from functions.wfr_settings import *
from functions.notebook_functions import *

# tibanna = Tibanna(env=env)
my_env = 'data'
my_auth = get_key('koray_data')

# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# #get admin key from s3
# my_env = 'fourfront-webdev'
# my_auth = ff_utils.get_authentication_with_server({}, ff_env = my_env)



set_url = '/search/?experiments_in_set.experiment_type=CUT%26RUN&type=ExperimentSetReplicate&limit=all&status=pre-release&status=released&status=released%20to%20project'
all_sets = ff_utils.search_metadata(set_url , key=my_auth)
counter = 0
completed = 0
completed_acc = []

run_sets = [i for i in all_sets if "CHIP_Pipeline_0.2.5"  not in i.get('completed_processes', [])]
print(len(all_sets), 'total number of sets')
print(len(all_sets)-len(run_sets), 'sets completed')
print(len(run_sets), 'ready for processing')

51 total number of sets
0 sets completed
51 ready for processing


In [9]:
run_wfr = True
add_pc = False
add_tag = False


def get_chip_files(replicate_exps, my_auth):
    files = []
    obj_key = []
    paired = ""
    for i in replicate_exps:
        exp = i['replicate_exp']['uuid']
        exp_resp = ff_utils.get_metadata(exp, my_auth)
        # get exp files
        exp_files = exp_resp['files']
        print('all files', [i['uuid'] for i in exp_files])
        b_index = int(i['bio_rep_no']) - 1
        t_index = int(i['tec_rep_no']) - 1
        for a_file in exp_files:
            print(a_file['uuid'], a_file['accession'])
            f_t = []
            o_t = []
            file_resp = ff_utils.get_metadata(a_file['uuid'], my_auth)
            # get pair end no
            pair_end = file_resp.get('paired_end')
            if pair_end == '2':
                paired = 'paired'
                print('skipped 2')
                continue
            # get paired file
            paired_with = ""
            relations = file_resp.get('related_files')
            if not relations:
                pass
            else:
                for relation in relations:
                    if relation['relationship_type'] == 'paired with':
                        paired = 'paired'
                        paired_with = relation['file']['uuid']
            # decide if data is not paired end reads
            if not paired_with:
                if not paired:
                    paired = 'single'
                else:
                    if paired != 'single':
                        print('inconsistent fastq pair info')
                        continue
                f_t.append(file_resp['uuid'])
                o_t.append(file_resp['display_title'])
            else:
                f2 = ff_utils.get_metadata(paired_with, my_auth)
                f_t.append(file_resp['uuid'])
                o_t.append(file_resp['display_title'])
                f_t.append(f2['uuid'])
                o_t.append(f2['display_title'])
            try:
                files[b_index]
            except:
                files.append([])
            files[b_index].append(f_t)
            #files[b_index][t_index] = f_t
            try:
                obj_key[b_index]
            except:
                obj_key.append([])
            obj_key[b_index].append(o_t)
            #obj_key[b_index][t_index] = o_t
            print('ind file', f_t)
        print('files are', files)
        print()
    return files, obj_key, paired


for a_set in run_sets: 
    print()
    counter += 1
    # some feature to extract from each set
    control = ""  # True or False (True if set in scope is control)
    control_set = ""  # None (if no control exp is set), or the control experiment for the one in scope
    target_type = "" # Histone or TF (or None for control)
    paired = "" # single or paired
    organism = ""
    
    # pass attributions to new objects
    attributions = None
    
    replicate_exps = a_set['replicate_exps']
    replicate_exps = sorted(replicate_exps, key = lambda x: [x['bio_rep_no'], x['tec_rep_no']])

    # get organism, target and control from the first replicate
    f_exp = replicate_exps[0]['replicate_exp']['uuid']
    f_exp_resp = ff_utils.get_metadata(f_exp, key = my_auth)
    control, control_set, target_type, organism = get_chip_info(f_exp_resp, my_auth)

    # Get list of files and object keys in triple nesting
    # sample = [[[b1t1,b1t1],[b1t2,b1t2]],[[b2t1,b2t1],[b2t2,b2t2]]]
    files, obj_key, paired = get_chip_files(replicate_exps, my_auth)
    
    print(a_set['accession'], organism, paired, control, control_set)  
    print(files)
    break
    
    if organism not in ['mouse', 'human']:
        print('orgamism not ready', organism)
        continue

    attributions = get_attribution(ff_utils.get_metadata(files[0][0][0], key = my_auth))
    
    if not control:
        if not target_type:
            print('set is not control, but missing target type, skipping')
            continue

    ta = []
    taxcor = []
    # check for step1, and start if missing
    step1_status = 'Done'
    for it in range(len(files)):
        run_name = a_set['accession'] + '_B' + str(it+1)
        print(run_name, end = ' status: ')
        bio_rep_files = [files[it]]
        bio_rep_obj = [obj_key[it]]
        if control:
            step1_result = get_wfr_out_file(bio_rep_files[0][0][0], 'encode-chipseq-aln-ctl 0.2.5', my_auth)
            print(step1_result['status'])
            if step1_result['status'] == 'complete':
                if add_pc:
                    add_chip_processed_files_exp([step1_result['chip.first_ta_ctl']], my_auth)
            elif step1_result['status'] == 'running':
                step1_status = 'Incomplete'
            else:
                step1_status = 'Incomplete'                
                if run_wfr:
                    print('starting run')
                    run_missing_chip1(control, step_settings('encode-chipseq-aln-ctl', organism, attributions), 
                                      organism, 'tf', paired, bio_rep_files, bio_rep_obj, my_env, my_auth, run_name)
        else:
            step1_result = get_wfr_out_file(bio_rep_files[0][0][0], 'encode-chipseq-aln-chip 0.2.5', my_auth) 
            print(step1_result['status'])
            if step1_result['status'] == 'complete':
                ta.append(step1_result['chip.first_ta'])
                taxcor.append(step1_result['chip.first_ta_xcor'])
                if add_pc:
                    add_chip_processed_files_exp([step1_result['chip.first_ta']], my_auth)
            elif step1_result['status'] == 'running':
                step1_status = 'Incomplete'
            else:
                step1_status = 'Incomplete'
                if run_wfr:
                    print('starting run')
                    run_missing_chip1(control, step_settings('encode-chipseq-aln-chip', organism, attributions), 
                                      organism, target_type, paired, bio_rep_files, bio_rep_obj, my_env, my_auth, run_name)

                    
    if step1_status != 'Done':
        continue
        
    # add pc files and tag for control
    if control:
        #add competed flag to set
        if add_tag:
            ff_utils.patch_metadata({"completed_processes":["CHIP_Pipeline_0.2.5"]}, obj_id=a_set['accession'] , key=my_auth)
        # add processed files to set
        continue
    
    print('set ready for part2')
    # check if control is ready
    #cont_set_info = ff_utils.get_metadata(control_set, key=my_key)
    
            
    


  


all files ['a7980683-3c11-4b53-8848-1efdf4cd89ef', '7fb0a920-e6f8-4a6b-9b35-123bd94618dd', 'a6a6ab8a-3a42-4680-8e0a-77420ff1a6f9', 'b3715bbf-478e-4ef9-b52d-2d8334c26fca', '7e9f797c-7d90-4a6e-8edd-ca06e7078da4', 'efa65615-20e8-4232-a531-14a3f607e139']
a7980683-3c11-4b53-8848-1efdf4cd89ef 4DNFISUSBDGO
ind file ['a7980683-3c11-4b53-8848-1efdf4cd89ef', '7fb0a920-e6f8-4a6b-9b35-123bd94618dd']
7fb0a920-e6f8-4a6b-9b35-123bd94618dd 4DNFI5S8S7EP
skipped 2
a6a6ab8a-3a42-4680-8e0a-77420ff1a6f9 4DNFILAYBB16
ind file ['a6a6ab8a-3a42-4680-8e0a-77420ff1a6f9', 'b3715bbf-478e-4ef9-b52d-2d8334c26fca']
b3715bbf-478e-4ef9-b52d-2d8334c26fca 4DNFIJCVPSP4
skipped 2
7e9f797c-7d90-4a6e-8edd-ca06e7078da4 4DNFIBIPUMNT
ind file ['7e9f797c-7d90-4a6e-8edd-ca06e7078da4', 'efa65615-20e8-4232-a531-14a3f607e139']
efa65615-20e8-4232-a531-14a3f607e139 4DNFIKXLPRKD
skipped 2
files are [[['a7980683-3c11-4b53-8848-1efdf4cd89ef', '7fb0a920-e6f8-4a6b-9b35-123bd94618dd'], ['a6a6ab8a-3a42-4680-8e0a-77420ff1a6f9', 'b3715bbf-478


3 4DNESH4UTRNL DpnII mouse 800  -  4DNEX4KRGMAQ is missing Part2 - Fails at runtaskawsem
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:hi-c-processing-bam_4DNEX4KRGMAQ9b484528-5651-4d72-a271-a9b37a42ab05



In [ ]:
# Move files from opc to pc
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.wfr import *

recipe_no = 1
exp_type, step3 = recipe[recipe_no]
action = False
move_title = 'HiC Processing Pipeline - Preliminary Files'

set_url = '/search/?'+ \
          '&'.join(['experiments_in_set.experiment_type='+i for i in exp_type])+ \
          '&type=ExperimentSetReplicate&limit=all' + \
          '&status=released&status=released%20to%20project'

print(set_url)
set_url = '/search/?award.project=4DN&experimentset_type=replicate&lab.display_title=Chuck+Murry%2C+UW&status=pre-release&type=ExperimentSetReplicate'

# exp
# set_url = '/search/?'+ \
#           '&'.join(['experiment_type='+i for i in exp_type])+ \
#           '&type=Experiment&limit=all' + \
#           '&status=released&status=released%20to%20project'

all_sets = ff_utils.search_metadata(set_url , key=my_auth)

ready_sets_1 = [i for i in all_sets if "HiC_Pipeline_0.2.5" in i.get('completed_processes', [])]
print(len(ready_sets_1))
ready_sets_2 = []
for a_set in ready_sets_1:
    if a_set.get('other_processed_files'):
        print('a')
        print(a_set['accession'])
        if move_title in [i['title'] for i in a_set['other_processed_files']]:
            print('b')
            if a_set.get('processed_files'):
                print('c')
                print('WARN' ,a_set['accession'], 'has items in processed files, skipping ')
                continue
            else:
                ready_sets_2.append(a_set)
print(len(ready_sets_2), 'items are ready')

In [ ]:
# move other processed files to processed files field
action = True
def move_opc_to_pc(resp, move_title, con_key):
    opc = resp.get('other_processed_files')
    pc = resp.get('processed_files')
    # if processed_files field already has values, exit
    if pc:
        if opc:
            print('There are files in processed_files field, expected empty', resp['accession'])
            return False
        else:
            print('it is possible that move already happened, no opc but pc', resp['accession'])
    # see if there are other_processed_files to move
    if opc:
        titles = [i['title'] for i in opc]
        if move_title in titles:
            print(resp['accession'], 'files will move')
            move_item = [i for i in opc if i['title'] == move_title]
            assert len(move_item) == 1
            assert move_item[0]['type'] == 'preliminary'
            new_pc = move_item[0]['files']
            new_opc = [i for i in opc if i['title'] != move_title]
            # Time to patch
            patch_data = {}
            add_on = ""
            #if there is something left in opc, patch it, if not delete field
            if new_opc:
                patch_data['other_processed_files'] = opc
            else:
                add_on = 'delete_fields=other_processed_files'
            # patch with processed files
            patch_data['processed_files'] = new_pc
            if action:
                ff_utils.patch_metadata(patch_data, resp['uuid'], key = con_key, add_on = add_on)
                # update status of pc to status of set or exp
                release_files(resp['uuid'], new_pc, con_key)
            return True
        else:
            return False

        
set_w_apf = 0
exp_w_apf = 0
counter = 0
move_title = 'HiC Processing Pipeline - Preliminary Files'

print(len(ready_sets_2), 'experiment sets in scope')
for a_set in ready_sets_2:
    set_resp = ff_utils.get_metadata(a_set['uuid'],key=my_auth, add_on='frame=raw')
    counter += 1
    print(counter, set_resp['accession'])
    exps = set_resp['experiments_in_set']
    res =  move_opc_to_pc(set_resp, move_title, my_auth)
    if res:
        set_w_apf += 1
        print(set_resp['accession'], 'moved to pc')
  
    for exp in exps:
        exp_resp = ff_utils.get_metadata(exp, key=my_auth, add_on='frame=raw')
        res_e =  move_opc_to_pc(exp_resp,move_title,my_auth)
        if res_e:
            exp_w_apf += 1
            print(exp_resp['accession'], 'moved to pc')
    print()

print(set_w_apf)
print(exp_w_apf)